# RAG와 Memory Chain을 모두 적용한 챗봇 만들기를 위한 연습입니다🙂
# Chabots
 - llm에 대한 가장 인기있는 사용케이스
 - 오랜기간의 학습을 거치며, 관련 정보를 이용, 현재상황에 대한 대화를 나눌 수 있음

# Atchitecture
 - Chatbot을 디자인하기 위해서는 여러 요소를 고려해야함
 - 당연히 DB, Memory 등 모든 정보를 넣고 prompt를 제공하면 성능은 높아지지만 속도, 자원 등 trade-off를 생각해야함
 - 아래는 일반적인 chatbot의 아키텍처
 ![Chabot Architecture](./imgs/Chatbot_Architecture.png)



# Chatbot의 기본 구성요소
 - Chatmodels : 사용자 쿼리 및 제공 데이터를 가지고 텍스트를 생성할 LLM모델
 - PromptTemplates : 모델이 사용자 요구사항을 잘 받아들이게 하고, 출력물의 결과를 사용자가 원하는 형태로 만들기 위한 template
 - ChatHistory : 사용자와의 대화 기록을 저장, 다음 답변 시 이전의 대화를 바탕으로 답변을 생성할 수 있음
 - Retrievers : 사용자의 쿼리를 통해 적정한 데이터를 찾아서 Chatmodels에게 제공해주는 역할
 

In [87]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [88]:
# 우선은 GPT 모델을 사용해봅니다
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
                   base_url="http://localhost:1234/v1",
                   api_key="lm-studio",
                   model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
                   temperature = 0.0,
                   )


In [89]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage as AIM

chat.invoke(
    [HumanMessage(content = "Translate this sentence from English to French: I love programming.")])

#질의 형식 보기


AIMessage(content='Je adore le programme.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 6, 'total_tokens': 12}, 'model_name': 'teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3f24db3-ee62-470d-919c-f3002a2ca7cd-0')

# 위 상태의 챗봇은 이전 대화를 기억하지 못합니다.
 - 가장 간단한 방식으로 이전의 대화를 기억하게끔 chatbot을 구성해보겠습니다.

Human Message와 AIMessage가 섞여있을 때 Chatbot은 가장 최근의 Human Message에 대해 대화를 진행합니다.

In [90]:
AIMessage = chat.invoke(
    [
        HumanMessage(content = "Translate this sentence from English to French: I love programming."),
        AIM(content = "J'aime le codage"),
        HumanMessage(content = "What did you just say?")
    ]
)

AIMessage.content

'Je viens de dire que j\'aime le codage, ce qui signifie en français "I love programming".'

# PromptTemplate을 주면 Chatbot이 더욱 효율적으로 과제를 알아들을 수 있습니다.
- 우리는 model에 template을 넣는 방식으로 이를 해결할 수 있습니다

In [91]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# prompt는 모델에 잘 입력될 수 있는 형태로 정의해야합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        # MessagesPlaceholder는 대화의 이전 메시지들을 포함하는 변수입니다. 이를 통해 챗봇은 이전 대화의 맥락을 이해하고, 연속적인 대화를 진행할 수 있습니다. 'messages'라는 변수 이름으로 이전 메시지들을 참조하게 됩니다.
        MessagesPlaceholder(variable_name = "messages")
])
chain = prompt | chat

# Message들 자체를 메타데이터로 주어 기억하게 할 수도 있습니다.
- MessagePlaceholder는 대화의 이전 메시지들을 포함하는 변수입니다. 이를 통해 챗봇은 이전 대화의 맥락을 이해하고, 연속적인 대화를 진행할 수 있습니다. 'messages'라는 변수 이름으로 이전 메시지들을 참조하게 됩니다.



In [92]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."),
            AIM(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
).content

'Je viens de dire que j\'adorer la programmation, ce qui signifie en français "I love programming".'

# Message history
- 채팅 기록을 관리하기 위한 간편한 방법으로, 채팅 메시지를 저장하고 불러오는 역할을 하는 MessageHistory 클래스를 사용할 수 있습니다. 
- 다양한 데이터베이스에 메시지를 지속적으로 저장하는 많은 내장 메시지 기록 통합 기능이 있지만, 이 빠른 시작 가이드에서는 메모리 내에서 작동하는 데모 메시지 기록인 ChatMessageHistory를 사용하겠습니다.


In [93]:
#ChatMessagehistory를 이용하면 간단한 방식으로 Memory를 관리할 수 있습니다 가령,
from langchain.memory import ChatMessageHistory
#인스턴스 생성
chathistory = ChatMessageHistory()
#add_user_message()메서드를 이용한 유저메시지 삽입
chathistory.add_user_message('hi')
#add_ai_message()메서드를 이용한 유저메시지 삽입
chathistory.add_user_message("what's up?")

chathistory.messages

[HumanMessage(content='hi'), HumanMessage(content="what's up?")]

# 생성한 chatmessagehistory를 chain의 Messageplaceholder에 넣어 대화를 진행할 수 있습니다.

In [96]:
chathistory.add_user_message(
    "Summarize the conversation in bullet points."
)

response = chain.invoke({"messages": chathistory.messages})

response.content

'- Tu es un assistant utile\n- Tu réponds aux questions au mieux de tes capacités\n- Tu as besoin d\'aide pour traduire une phrase en français\n- La phrase est "J\'aimer le programme"\n- Tu veux résumer la conversation sous forme de points saillants'

# 위에서 만든 챗봇을 이용해 RAG를 적용해보겠습니다.

In [85]:
# 여기서는 webbaseloader를 이용해 인터넷에 있는 문서를 가져오도록 하겠습니다.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [86]:
# 가져온 데이터는 대부분의 경우 매우 많은 양의 데이터를 가지고 있습니다.
data

[Document(page_content='\n\n\n\n\nGet started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it\'s not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace‚ÄãWe provide multiple ways to log 

# Splitter
많은 양의 문서를 한번에 context로 제공하는 것은 많은 자원을 요구하거나 불가능한 경우가 많습니다.